<a href="https://colab.research.google.com/github/cswcjt/Fastcampus-ML/blob/main/imbalance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

-- Imbalanced data problem: 비대칭 데이터 문제

    - 데이터 클래스 비율이 너무 차이가 나면(highly-imbalanced data) 단순히 우세한 클래스를 택하는 모형의 정확도가 높아지므로 모형의 성능판별이 어려워진다. 즉, 정확도(accuracy)가 높아도 데이터 갯수가 적은 클래스의 재현율(recall-rate)이 급격히 작아지는 현상이 발생할 수 있다.

-- Imbalanced-Learn methodology: 해결 방법 

    - 비대칭 데이터는 다수 클래스 데이터에서 일부만 사용하는 언더 샘플링이나 소수 클래스 데이터를 증가시키는 오버 샘플링을 사용하여 데이터 비율을 맞추면 정밀도(precision)가 향상된다. 

    - 오버샘플링(Over-Sampling)

    - 언더샘플링(Under-Sampling)

    - 복합샘플링(Combining Over-and Under-Sampling)

-- under sampling 

    - RandomUnderSampler: 무작위로 데이터를 없애는 단순 샘플링

    - TomekLinks: 
        - 1) 토멕링크(Tomek’s link)란 서로 다른 클래스에 속하는 한 쌍의 데이터 (𝑥+,𝑥−)로 서로에게 더 가까운 다른 데이터가 존재하지 않는 것이다. 
        - 2) 토멕링크를 찾은 다음 그 중에서 다수 클래스에 속하는 데이터를 제외하는 방법

    - CondensedNearestNeighbour(CNN): 
        - CNN(Condensed Nearest Neighbour) 방법은 1-NN 모형으로 분류되지 않는 데이터만 남기는 방법이다. 선텍된 데이터 집합을 𝑆라고 하자.
        - 1) 소수 클래스 데이터를 모두 𝑆에 포함시킨다.
        - 2) 다수 데이터 중에서 하나를 골라서 가장 가까운 데이터가 다수 클래스이면 포함시키지 않고 아니면 𝑆에 포함시킨다.
        - 3) 더이상 선택되는 데이터가 없을 때까지 3를 반복한다.

    - OneSidedSelection: 
        - TomekLinks + CNN 

    - EditedNearestNeighbours(ENN): 
        - 다수 클래스 데이터 중 가장 가까운 k(n_neighbors)개의 데이터가 모두(kind_sel="all") 또는 다수(kind_sel="mode") 다수 클래스가 아니면 삭제하는 방법이다. 소수 클래스 주변의 다수 클래스 데이터는 사라진다.

    - NeighbourhoodCleaningRule: 
        -  CNN + ENN

    
-- over sampling

    - RandomOverSampler
        - Random Over Sampling은 소수 클래스의 데이터를 반복해서 넣는 것(replacement)이다. 가중치를 증가시키는 것과 비슷하다. 

    - ADASYN(Adaptive Synthetic Sampling Approach for Imbalanced Learning)
        - 소수 클래스 데이터와 그 데이터에서 가장 가까운 k개의 소수 클래스 데이터 중 무작위로 선택된 데이터 사이의 직선상에 가상의 소수 클래스 데이터를 만드는 방법이다.

    - SMOTE(Synthetic Minority Over-sampling Technique)
        - ADASYN 방법처럼 데이터를 생성하지만 생성된 데이터를 무조건 소수 클래스라고 하지 않고 분류 모형에 따라 분류한다.

-- 복합 샘플링

    - SMOTE+ENN
        - SMOTE+ENN 방법은 SMOTE(Synthetic Minority Over-sampling Technique) 방법과 ENN(Edited Nearest Neighbours) 방법을 섞은 것이다. 
    - SMOTE+Tomek
        - SMOTE+Tomek 방법은 SMOTE(Synthetic Minority Over-sampling Technique) 방법과 토멕링크 방법을 섞은 것이다.

-- 주의 사항

    - train & test datasets의 포퍼먼스를 주시해야한다. 
        - to gain insight into the impact of the method, it is a good idea to monitor the performance on both train and test datasets after oversampling and compare the results to the same algorithm on the original dataset.

    - 클래스의 분포의 skewness가 높은지 확인이 필요하다. 
        - The increase in the number of examples for the minority class, especially if the class skew was severe, can also result in a marked increase in the computational cost when fitting the model, especially considering the model is seeing the same examples in the training dataset again and again.

    - Pipeline 사용시
        - A traditional scikit-learn Pipeline cannot be used; instead, a Pipeline from the imbalanced-learn library can be used

-- 왜도와 첨도

    - 왜도 (Skewness) : 분포의 비대칭도
        - 정규분포 = 왜도 0
        - 왼쪽으로 치우침 = 왜도 > 0
        - 오른쪽으로 치우침 = 왜도 < 0
    - 첨도 (Kurtosis) : 확률분포의 뾰족한 정도
        - 정규분포 = 첨도 0(Pearson 첨도 = 3)
        - 위로 뾰족함 = 첨도 > 0(Pearson 첨도 >3)
        - 아래로 뾰족함 = 첨도 < 0 (Pearson 첨도 < 3) 


 

Preprocessing에 들어갈 imbalance 함수 만들자

In [ ]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount('/content/drive')
base_path = "/content/drive/MyDrive/fastcamp/datas/open/"
train = pd.read_csv(base_path + "train.csv", encoding='cp949')
test = pd.read_csv(base_path + "test.csv", encoding='cp949')
submission = pd.read_csv(base_path + "sample_submission.csv", encoding='cp949')
print("Train shape : ", train.shape)
print("Test shape : ", test.shape)

Mounted at /content/drive
Train shape :  (14095, 54)
Test shape :  (6041, 19)


In [ ]:
from collections import Counter # 샘플결과 확인
from sklearn.model_selection import train_test_split as tts # 트테트테
from sklearn.decomposition import PCA # 차원축소
from sklearn.ensemble import RandomForestClassifier as RFC # 모델선택
from sklearn.metrics import classification_report # 성과지표
from imblearn.under_sampling import * # 임벨런스
from imblearn.over_sampling import * # 임벨런스
from imblearn.combine import * # 임벨런스
from imblearn.pipeline import Pipeline # 파이프라인구축

In [ ]:
train = train.fillna(0)

In [ ]:
X_imb = train.drop(columns = ["Y_LABEL", "ID", "COMPONENT_ARBITRARY"])
y_imb = train.Y_LABEL

In [ ]:
base_path = "/content/drive/MyDrive/fastcamp/datas/open/"
train = pd.read_csv(base_path + "train.csv", encoding='cp949')
test = pd.read_csv(base_path + "test.csv", encoding='cp949')
submission = pd.read_csv(base_path + "sample_submission.csv", encoding='cp949')

class oil:
    def __init__(self):
        base_path = '/content/drive/MyDrive/fastcamp/datas/open/'
        self.load_path = base_path #+ 'data/'
        self.save_path = base_path #+ 'submission/'
        self.train = pd.read_csv(self.load_path + 'train.csv')
        self.test = pd.read_csv(self.load_path + 'test.csv')
        self.submission = pd.read_csv(self.load_path + 'sample_submission.csv')

        self.X = train.drop(columns=['ID', 'Y_LABEL'])
        self.y = train.Y_LABEL
        self.X_test = test.drop(columns=['ID'])
        #self.X2
    
    def reset_data(self, mode: str=None):
        if mode is None:
            self.train = pd.read_csv(self.load_path + 'train.csv')
            self.test = pd.read_csv(self.load_path + 'test.csv')
        elif mode == 'train':
            self.train = pd.read_csv(self.load_path + 'train.csv')
        elif mode == 'test':
            self.test = pd.read_csv(self.load_path + 'test.csv')

In [ ]:
# 모든 imbalance sampler 담고있는 df 만든다. 
sampling_method_info = pd.DataFrame(
    {"under_sampling" : [RandomUnderSampler(),
                        TomekLinks(), 
                        CondensedNearestNeighbour(), 
                        OneSidedSelection(), 
                        EditedNearestNeighbours(),
                        NeighbourhoodCleaningRule()],

    "over_sampling" : [RandomOverSampler(),
                       ADASYN(),
                       NeighbourhoodCleaningRule(),
                       np.NaN,
                       np.NaN,
                       np.NaN],

    "hybrid_samping" : [SMOTEENN(),
                        SMOTETomek(),
                        np.NaN,
                        np.NaN,
                        np.NaN,
                        np.NaN]
     }
)

In [ ]:
first_ins = oil()
X_imb = first_ins.X
X_imb = X_imb.drop(columns = "COMPONENT_ARBITRARY").fillna(0)
y_imb = first_ins.y.fillna(0)

In [ ]:
def call_samping_method(sampling_method_info_df, one_of_columns: str = None) :
    balanced_list = []
    for sampler in sampling_method_info_df[one_of_columns] : 
        balanced_list.append(sampler.fit_resample(X_imb, y_imb))
    print(balanced_list)

### classification_report -> str에서 df로 바꾸면 편할듯
def pipine(sampling_method = None, dimensionality = None, model = None) : 
    pipeline = Pipeline([('sampling_method', sampling_method), ('dimensionality', dimensionality), ('model', model)]) # sampling method, dimensionality, model
    X_train, X_test, y_train, y_test = tts(X_samp, y_samp, random_state=42)
    pipeline.fit(X_train, y_train) 
    y_hat = pipeline.predict(X_test)
    #print(type(classification_report(y_test, y_hat)))
    return classification_report(y_test, y_hat)

In [ ]:
after_sampling = call_samping_method(sampling_method_info, "under_sampling")
after_sampling

[(      ANONYMOUS_1  YEAR  SAMPLE_TRANSFER_DAY  ANONYMOUS_2  AG  AL    B  BA  \
0            1416  2015                    7          616   0   1    2   3   
1            4990  2009                    3          200   0   4    8   0   
2            1388  2022                    2          393   0   3   12   0   
3            2522  2007                    5          620   0   2    4   1   
4            1913  2010                    5          200   0   1    2   0   
...           ...   ...                  ...          ...  ..  ..  ...  ..   
2401         7789  2012                    4          200   0   3   41   3   
2402         3060  2010                    2          200   0   2    3   0   
2403         2036  2010                    5          200   0  37    6   1   
2404         1637  2008                   16          487   0  11   99   2   
2405         1552  2007                    5          375   0   4  111   0   

      BE    CA  ...  U50   U25   U20  U14   U6   U4  V  V100 

In [ ]:
pca = PCA()
rfc = RFC()
rmu = RandomUnderSampler()
pipine(rmu, pca, rfc)

In [ ]:
# under_sampling
X_samp, y_samp = RandomUnderSampler().fit_resample(X_imb, y_imb)
X_samp, y_samp = TomekLinks().fit_resample(X_imb, y_imb)
#X_samp, y_samp = CondensedNearestNeighbour().fit_resample(X_imb, y_imb) # -> 오래걸림 8분에서 멈춤
X_samp, y_samp = OneSidedSelection().fit_resample(X_imb, y_imb)
X_samp, y_samp = EditedNearestNeighbours(kind_sel="all", n_neighbors=5).fit_resample(X_imb, y_imb)
X_samp, y_samp = NeighbourhoodCleaningRule(kind_sel="all", n_neighbors=5).fit_resample(X_imb, y_imb)

# over_sampling
X_samp, y_samp = RandomOverSampler(random_state=0).fit_resample(X_imb, y_imb)
X_samp, y_samp = ADASYN(random_state=0).fit_resample(X_imb, y_imb)
X_samp, y_samp = SMOTE(random_state=4).fit_resample(X_imb, y_imb)

# hybrid_samping
X_samp, y_samp = SMOTEENN(random_state=0).fit_resample(X_imb, y_imb)
X_samp, y_samp = SMOTETomek(random_state=4).fit_resample(X_imb, y_imb)

In [ ]:
# pipe line 
def pipine(sampling_method = None, dimensionality = None, model = None) : 
    pipeline = Pipeline([('sampling_method', sampling_method), ('dimensionality', dimensionality), ('model', model)]) # sampling method, dimensionality, model
    X_train, X_test, y_train, y_test = tts(X_samp, y_samp, random_state=42)
    pipeline.fit(X_train, y_train) 
    y_hat = pipeline.predict(X_test)
    print(classification_report(y_test, y_hat))

In [ ]:
pca = PCA()
rfc = RFC()
rmu = RandomUnderSampler()
pipine(rmu, pca, rfc)